### RRT using `Panda`

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from mujoco_parser import MuJoCoParserClass
from rrt import RapidlyExploringRandomTreesStarClass
from util import rpy2r,r2rpy
from scipy.spatial.distance import cdist
np.set_printoptions(precision=2,suppress=True,linewidth=100)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("networkx version:[%s]"%(nx.__version__))
print ("MuJoCo version:[%s]"%(mujoco.__version__))

networkx version:[3.0]
MuJoCo version:[2.3.2]


### Parse `Panda`

In [2]:
xml_path = '../model/panda/franka_panda_w_objs.xml'
env = MuJoCoParserClass(name='Panda',rel_xml_path=xml_path,VERBOSE=False)
# Initialize object locations
np.random.seed(seed=1)
obj_names = env.get_body_names(prefix='obj_')
n_obj = len(obj_names)
colors = np.array([plt.cm.gist_rainbow(x) for x in np.linspace(0,1,n_obj)])
colors[:,3] = 1.0
for obj_idx,obj_name in enumerate(obj_names):
    env.model.body(obj_name).ipos = [-1.0,0.1*obj_idx,0.01]
    env.model.body(obj_name).iquat = [0,0,0,1]
    env.model.geom(obj_name).rgba = colors[obj_idx] # color
# Add transparency to geoms (excluding ground)
for geom_name in env.geom_names:
    if geom_name == 'ground': continue
    env.model.geom(geom_name).rgba[3] = 0.8
# Set objects
body_name = 'obj_box_01'
geom_name = env.geom_names[env.model.body(body_name).geomadr[0]]
pos_z,obj_height = 1.0,0.8 # object height and z-position
env.model.geom(geom_name).pos  = np.array([0.0,0.0,0.0])
env.model.geom(geom_name).size = np.array([0.05,0.05,obj_height/2])
env.model.body(body_name).pos  = np.array([0.0,0.0,pos_z])
env.model.body(body_name).ipos = np.array([0.5,0.0,obj_height/2]) # position
# Check env.model.body(body_name) / env.model.geom(geom_name)
print ("Done.")

Done.


### Get initial and final poses by solving IK

In [3]:
env.init_viewer(viewer_title='Spawn Objects',viewer_width=1200,viewer_height=800,viewer_hide_menus=True)
env.update_viewer(azimuth=80,distance=2.5,elevation=-30,lookat=[0,0,1.5])
env.update_viewer(VIS_TRANSPARENT=True,VIS_CONTACTPOINT=False,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]))
env.update_viewer(VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])
env.reset() # reset
q_rev_init = env.get_q(joint_idxs=env.rev_joint_idxs) # get initial joint position

# Open gripper (prismatic joints)
env.forward(q=np.array([0,0,0,0,0,90,0])*np.pi/180.0,joint_idxs=env.rev_joint_idxs)
env.forward(q=[0.04,-0.04],joint_idxs=env.pri_joint_idxs) # open gripper

# Set IK for the initial pose
ik_body_name = 'panda_eef'
p_trgt,R_trgt = np.array([0.6,-0.5,1.1]),rpy2r(np.array([-180,0,90])*np.pi/180.0)
env.forward(q=q_rev_init,joint_idxs=env.rev_joint_idxs)
while (env.tick < 10000) and env.is_viewer_alive():
    q,err = env.onestep_ik(
        body_name=ik_body_name,p_trgt=p_trgt,R_trgt=R_trgt,IK_P=True,IK_R=True,
        joint_idxs=env.rev_joint_idxs,stepsize=1,eps=1e-1,th=5*np.pi/180.0)
    env.render(render_every=5)
    if np.linalg.norm(err) < 1e-6: break
q_init = env.get_q(joint_idxs=env.rev_joint_idxs)

# Set IK for the final pose
ik_body_name = 'panda_eef'
p_trgt,R_trgt = np.array([0.6,0.5,1.1]),rpy2r(np.array([-180,0,90])*np.pi/180.0)
env.forward(q=q_rev_init,joint_idxs=env.rev_joint_idxs)
while (env.tick < 10000) and env.is_viewer_alive():
    q,err = env.onestep_ik(
        body_name=ik_body_name,p_trgt=p_trgt,R_trgt=R_trgt,IK_P=True,IK_R=True,
        joint_idxs=env.rev_joint_idxs,stepsize=1,eps=1e-1,th=5*np.pi/180.0)
    env.render(render_every=5)
    if np.linalg.norm(err) < 1e-6: break
q_final = env.get_q(joint_idxs=env.rev_joint_idxs)

# Close viewer
env.close_viewer()
print ("Done.")

Done.


### RRT

In [6]:
RRT = RapidlyExploringRandomTreesStarClass(
    name='RRT-Star-Panda',
    point_min=env.joint_ranges[env.rev_joint_idxs][:,0],
    point_max=env.joint_ranges[env.rev_joint_idxs][:,1],
    goal_select_rate=0.1,
    steer_len_max=np.deg2rad(30),search_radius=np.deg2rad(45),norm_ord=2,
    n_node_max=10000,TERMINATE_WHEN_GOAL_REACHED=False)
# Set root and goal points
RRT.init_rrt_star(point_root=q_init,point_goal=q_final)
print ("[%s] ready."%(RRT.name))

[RRT-Star-Panda] ready.


### Define `is_point_feasible` and `is_point_to_point_connectable`

In [7]:
def is_point_feasible(env,point_sample):
    q_bu = env.get_q(joint_idxs=env.rev_joint_idxs)
    env.forward(q=[0.04,-0.04],joint_idxs=env.pri_joint_idxs) # open gripper
    env.forward(q=point_sample,joint_idxs=env.rev_joint_idxs)
    p_contacts,f_contacts,geom1s,geom2s = env.get_contact_info(must_include_prefix='panda')
    # nv.forward(q=q_bu,joint_idxs=env.rev_joint_idxs)
    if len(p_contacts) == 0: return True
    else: return False
def is_point_to_point_connectable(env,point_fr,point_to):
    return True
print ("Ready.")

Ready.


### RRT Loop

In [ ]:
# Init viewer (for debugging purposes)
env.init_viewer(viewer_title='Spawn Objects',viewer_width=1200,viewer_height=800,viewer_hide_menus=True)
env.update_viewer(azimuth=80,distance=2.5,elevation=-30,lookat=[0,0,1.5])
env.update_viewer(VIS_TRANSPARENT=True,VIS_CONTACTPOINT=False,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]))
env.update_viewer(VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])
env.reset() # reset
# Zero-pose with gripper open
env.forward(q=np.array([0,0,0,0,0,90,0])*np.pi/180.0,joint_idxs=env.rev_joint_idxs)
env.forward(q=[0.04,-0.04],joint_idxs=env.pri_joint_idxs) # open gripper
while (RRT.get_n_node() <= RRT.n_node_max) and RRT.increase_loop_cnt() and env.is_viewer_alive():
    
    # Randomly sample a point 
    while True:
        if np.random.rand() <= RRT.goal_select_rate: point_sample = RRT.point_goal 
        else: point_sample = RRT.sample_point() # random sampling
        if is_point_feasible(env,point_sample): break

    # Get the nearest node ('node_nearest') to 'point_sample' from the tree
    node_nearest = RRT.get_node_nearest(point_sample)
    
    # Steering towards 'point_sample' to get 'point_new'
    point_new,cost_new = RRT.steer(node_nearest,point_sample)
    if point_new is None: continue # if the steering point is feasible

    # Rewiring
    if is_point_feasible(env,point_new) \
        and is_point_to_point_connectable(
        env,RRT.get_node_point(node_nearest),point_new):
        
        # Assign 'node_min' and initialize 'cost_min' with 'cost_new'
        node_min = node_nearest.copy()
        cost_min = cost_new
        
        # Select a set of nodes near 'point_new' => 'nodes_near'
        nodes_near = RRT.get_nodes_near(point_new)

        # For all 'node_near' find 'node_min'
        for node_near in nodes_near:
            point_near,cost_near = RRT.get_node_point_and_cost(node_near)
            if is_point_to_point_connectable(env,point_near,point_new):
                cost_prime = cost_near + RRT.get_dist(point_near,point_new)
                if cost_prime < cost_min:
                    cost_min = cost_near + RRT.get_dist(point_near,point_new)
                    node_min = node_near
                    
        # Add 'node_new' and connect it with 'node_min'
        node_new = RRT.add_node(point=point_new,cost=cost_min,node_parent=node_min)
        
        # New node information for rewiring
        point_new,cost_new = RRT.get_node_point_and_cost(node_new)
        
        # Rewire
        for node_near in nodes_near:
            point_near,cost_near = RRT.get_node_point_and_cost(node_near)
            cost_check = cost_new+RRT.get_dist(point_near,point_new)
            if (cost_check < cost_near) and \
                is_point_to_point_connectable(env,point_near,point_new):
                RRT.replace_node_parent(node=node_near,node_parent_new=node_new)

    # Re-update cost of all nodes
    # RRT.update_nodes_cost(VERBOSE=False)
    
    # Debug plot
    env.forward(q=point_new,joint_idxs=env.rev_joint_idxs)
    env.render(render_every=10)

    # Terminate condition
    dist_to_goal = RRT.get_dist_to_goal()
    if (dist_to_goal < 1e-6) and RRT.TERMINATE_WHEN_GOAL_REACHED:
        print ("Goad reached. n_node:[%d] dist_to_goal:[%.4f]"%
               (RRT.get_n_node(),dist_to_goal))
        break
        
# Close viewer
env.close_viewer()        
print ("Done.")

### Play RRT results

In [ ]:
path_to_goal,path_node_list = RRT.get_path_to_goal()
for node_idx,node in enumerate(path_node_list):
    print ("[%d/%d] node:[%d] point:%s cost:[%.2f]"%
           (node_idx,len(path_node_list),node,RRT.get_node_point(node),
            RRT.get_node_cost(node)))

In [ ]:
env.init_viewer(viewer_title='Spawn Objects',viewer_width=1200,viewer_height=800,viewer_hide_menus=True)
env.update_viewer(azimuth=80,distance=2.5,elevation=-30,lookat=[0,0,1.5])
env.update_viewer(VIS_TRANSPARENT=True,VIS_CONTACTPOINT=False,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]))
env.update_viewer(VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])
# env.reset() # reset

for q in path_to_goal:
    for _ in range(100):
        env.forward(q=q,joint_idxs=env.rev_joint_idxs)
        env.render(render_every=1)
env.close_viewer() # close viewer
print ("Done.")